In [1]:


%%capture
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
import datetime as d 
load_dotenv()

# get the current directory
current_dir = os.getcwd()

# change the current working directory to the parent directory
os.chdir(os.path.dirname(current_dir))

path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))

USRPATH = "/Users/ducjeremyvu/dev/"


sys.path.append(Path().absolute().parent)
sys.path.append("/Users/ducjeremyvu/dev/wspace/nb")
sys.path.append(parent)
sys.path.append(...)

os.getcwd()


# from g_api import gservice


# Template


In [11]:
# Example usage
file1 = '/Users/ducjeremyvu/Downloads/ucmdownload/PR_202408_128/PR_202408_128_wd.csv'
file2 = '/Users/ducjeremyvu/Downloads/wd_uniq_2.csv'


def get_differences(file1, file2):
    original = pd.read_csv(file1, sep=';', dtype=str)
    new = pd.read_csv(file2, sep=';', dtype=str)

    # Convert the necessary columns to numeric, handling commas as decimal points
    original['VD_EBETRAG'] = pd.to_numeric(original['VD_EBETRAG'].str.replace(',', '.'), errors='coerce')
    new['VD_EBETRAG'] = pd.to_numeric(new['VD_EBETRAG'].str.replace(',', '.'), errors='coerce')

    sum_original = original['VD_EBETRAG'].sum()
    sum_new = new['VD_EBETRAG'].sum()

    # Define the columns used to uniquely match rows, including PNR, VERTNR, VD_DAT, and LA
    match_columns = ['PNR', 'VERTNR', 'VD_DAT', 'LA']

    # Check for duplicates in the key columns for both dataframes
    if original.duplicated(subset=match_columns).any():
        print("Warning: There are duplicate rows in the first file based on the unique key.")
    
    if new.duplicated(subset=match_columns).any():
        print("Warning: There are duplicate rows in the second file based on the unique key.")

    # Merge the two dataframes on the specified unique key columns
    merged = pd.merge(original, new, on=match_columns, suffixes=('_old', '_new'), how='outer', indicator=True)

    # Ensure the columns VD_EBETRAG_old and VD_EBETRAG_new are numeric
    merged['VD_EBETRAG_old'] = pd.to_numeric(merged['VD_EBETRAG_old'], errors='coerce')
    merged['VD_EBETRAG_new'] = pd.to_numeric(merged['VD_EBETRAG_new'], errors='coerce')

    # Filter the merged dataframe to identify differences
    # Case 1: Rows that exist in both but have different VD_EBETRAG
    # differences = merged[(merged['VD_EBETRAG_old'] != merged['VD_EBETRAG_new']) &
    #                      (merged['_merge'] == 'both')]

    differences = merged.loc[merged["_merge"] == "right_only"]

    # Create a result dataframe, exclude 'VD_EFAKT' for now
    result_df = pd.DataFrame()

    result_df['SATZART'] = differences['SATZART_new']
    result_df['FUNKTION'] = differences['FUNKTION_new']
    result_df['MAN'] = differences['MAN_new']
    result_df['AK'] = differences['AK_new']
    result_df['PNR'] = differences['PNR']
    result_df['NAME'] = differences['NAME_new']
    result_df['VORNAME'] = differences['VORNAME_new']
    result_df['VERTNR'] = differences['VERTNR']
    result_df['LA'] = differences['LA']
    result_df['VD_ETAGE'] = differences['VD_ETAGE_new']
    result_df["VD_ESTD"] =  ""
    result_df["VD_EFAKT"]  = ""
    # Process the 'VD_EBETRAG_diff' and other numeric columns
    result_df['VD_EBETRAG'] = (differences['VD_EBETRAG_new']).map('{:.2f}'.format).str.replace('.', ',')
    result_df['KST'] = differences['KST_new']
    result_df['KOSTART'] = differences['KOSTART_new']
    result_df['VD_KTR'] = differences['VD_KTR_new']
    result_df['VD_DAT'] = differences['VD_DAT']
    result_df['VD_ZDAT'] = differences['VD_ZDAT_new']
    result_df['VD_HER'] = "128_C"



    return result_df
    #return [sum_original, sum_new, sum_new - sum_original, merged.loc[merged["_merge"] == "right_only"]["VD_EBETRAG_new"].sum()]
    #return merged["_merge"].unique()
    

result = get_differences(file1, file2)





# result.to_csv('out/differences.csv', sep=';', index=False)



In [12]:
result

,SATZART,FUNKTION,MAN,AK,PNR,NAME,VORNAME,VERTNR,LA,VD_ETAGE,VD_ESTD,VD_EFAKT,VD_EBETRAG,KST,KOSTART,VD_KTR,VD_DAT,VD_ZDAT,VD_HER
13934,[VARTAB],INSERT,10246,201,A20984,NaN,NaN,11,104,NaN,,,"101,25",NaN,NaN,NaN,2024-08-07,2024-08-01,128_C
13935,[VARTAB],INSERT,10246,201,A20984,NaN,NaN,11,284,NaN,,,"10,65",NaN,NaN,NaN,2024-08-07,2024-08-01,128_C
13936,[VARTAB],INSERT,10246,201,A20984,NaN,NaN,11,706,NaN,,,"8,10",NaN,NaN,NaN,2024-08-07,2024-08-01,128_C
13937,[VARTAB],INSERT,10246,201,A40842,NaN,NaN,40,104,NaN,,,"111,38",NaN,NaN,NaN,2024-08-26,2024-08-01,128_C
13938,[VARTAB],INSERT,10246,201,A40842,NaN,NaN,40,284,NaN,,,"4,13",NaN,NaN,NaN,2024-08-26,2024-08-01,128_C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14541,[VARTAB],INSERT,10246,201,A84335,NaN,NaN,1,284,NaN,,,"3,75",NaN,NaN,NaN,2024-08-30,2024-08-01,128_C
14542,[VARTAB],INSERT,10246,201,A84627,NaN,NaN,1,104,NaN,,,"101,25",NaN,NaN,NaN,2024-08-31,2024-08-01,128_C
14543,[VARTAB],INSERT,10246,201,A84627,NaN,NaN,1,284,NaN,,,"3,75",NaN,NaN,NaN,2024-08-31,2024-08-01,128_C
14544,[VARTAB],INSERT,10246,201,A85052,NaN,NaN,1,104,NaN,,,"104,63",NaN,NaN,NaN,2024-08-29,2024-08-01,128_C


In [14]:
result_adj = result.copy()

result_adj["VD_EBETRAG"] = pd.to_numeric(result_adj["VD_EBETRAG"].str.replace(",", "."))

result_grouped = result_adj.groupby(["PNR","VERTNR"])["VD_EBETRAG"].sum().reset_index()

# result_grouped["half"] = (result_grouped["VD_EBETRAG"] / 2).round(2)

result_grouped


,PNR,VERTNR,VD_EBETRAG
0,A20984,11,120.00
1,A40842,40,115.51
2,A46930,2,289.00
3,A48874,31,210.00
4,A51579,4,141.07
...,...,...,...
92,A83137,4,93.00
93,A83443,1,1071.64
94,A84335,1,105.00
95,A84627,1,105.00


In [23]:
wd_column_order = [
    "SATZART",
    "FUNKTION",
    "MAN",
    "AK",
    "PNR",
    "NAME",
    "VORNAME",
    "VERTNR",
    "LA",
    "VD_ETAGE",
    "VD_ESTD",
    "VD_EFAKT",
    "VD_EBETRAG",
    "KST",
    "KOSTART",    
    "VD_KTR",
    "VD_DAT",
    "VD_ZDAT",
    "VD_HER",
    # "VD_HER_DATE",
    # "VD_PROZ",
    # "VD_KSTB",
    # "VD_USERID",
    # "VD_WERT",
    # "KST2_MAN",
    # "KST2_AK",
    # "VD_KALK",
#    "VD_TXT",
]

def add_abschlag_rows(dataframe):
    dataframe = dataframe.copy()
    dataframe["MAN"] = "10246"
    dataframe["AK"] = "102"
    dataframe["LA"] = "811"
    dataframe["VD_ZDAT"] = "2024-09-01"
    dataframe["SATZART"] = "[VARTAB]"
    dataframe["FUNKTION"] = "INSERT"
    dataframe["VD_HER"] = "128_C"
    dataframe["VD_DAT"] = ""
    dataframe["VD_ETAGE"] = ""
    dataframe["VD_EBETRAG"] = dataframe["VD_EBETRAG"].round(2).astype(str).str.replace(".",",")
    dataframe["KST"] = ""
    dataframe["KOSTART"] = ""
    dataframe["VD_KTR"] = ""
    dataframe["VD_ESTD"] = ""
    dataframe["VD_EFAKT"] = ""
    dataframe["NAME"] = ""
    dataframe["VORNAME"] = "" 
    dataframe["VD_HER_DATE"] = ""
    dataframe["VD_PROZ"] = ""
    dataframe["VD_KSTB"] = ""
    dataframe["VD_USERID"] = ""
    dataframe["VD_WERT"] = ""
    dataframe["KST2_MAN"] = ""
    dataframe["KST2_AK"] = ""
    dataframe["VD_KALK"] = ""

    return dataframe[wd_column_order]


result_abschlag = add_abschlag_rows(result_grouped)



In [28]:
export = pd.concat([result, result_abschlag], ignore_index=True)


export.to_csv('out/2409_studnets_differences.csv', sep=';', index=False)